#Data Engineer Challenge - Miguel Guerrero

obs: Se importan las librerias dentro del segmento de codigo de cada solucion con el objetivo que sea utilizado unicamente ahi a pesar que puede ser reutilizado en otro segmento
esto con el objetivo de aportar claridad a las librerias necesarias para el funcionamiento de la solucion

# Las top 10 fechas donde hay más tweets. 
Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
# Enfoque: Tiempo de ejecucion optimizado

Para este caso convertimos el archivo json a un dataframe de pandas por los beneficios que este tiene en la optimizacion de las operaciones con datos.
Como se esta priorizando el tiempo de ejecucion por sobre de la memoria.

In [ ]:
import pandas as pd
from datetime import datetime
from typing import List, Tuple

#Definimos la funcion que nos devolvera la lista de los 10 usuarios con mas tweets
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    
    # Cargamos los datos en un df de pandas
    df = pd.read_json(file_path, lines=True)
    
    # Convertimos la columna date a datetime de pandas y extraemos solo la fecha
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    # Extraemos el nombre de usuario de la columna user
    df['username'] = df['user'].apply(lambda x: x['username'])

    # Agrupamos por fecha y nombre de usuario y contamos el numero de tweets
    grouped = df.groupby(['date', 'username']).size().reset_index(name='count')
    
    # Ordenamos por fecha y numero de tweets de forma descendente
    grouped_sorted = grouped.sort_values(by=['date', 'count'], ascending=[False, False])
    
    # Obtenemos las top 10 fechas con mas tweets
    top_dates = grouped_sorted['date'].unique()[:10]
    
    # Obtenemos el usuario con mas tweets para cada una de las top 10 fechas
    result = []
    for date in top_dates:
        #Como ya esta ordenado el df, el primer usuario de cada fecha sera el que tiene mas tweets
        top_user_row = grouped_sorted[grouped_sorted['date'] == date].iloc[0]
        result.append((top_user_row['date'], top_user_row['username']))
    
    return result

q1_time("./farmers-protest-tweets-2021-2-4.json")